In [1]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-6a788e71-7ad2-fc90-0be4-4c7da76df7cb)


In [2]:

import pandas as pd
train_df = pd.read_csv("/content/drive/MyDrive/train.csv")
test_df = pd.read_csv("/content/drive/MyDrive/test.csv")
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [3]:
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [4]:
len(train_df)

7613

In [5]:
from sklearn.model_selection import train_test_split


In [6]:
train_sent,validation_sent,train_result,validation_result=train_test_split(
    train_df['text'].to_numpy(),
    train_df['target'].to_numpy(),
    test_size=0.2,
    random_state=42

)

In [7]:
len(train_sent),len(train_result)

(6090, 6090)

TextVectorization

In [8]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

In [9]:
textvectorizer=TextVectorization(
    max_tokens=None,
    standardize="lower_and_strip_punctuation",
    split="whitespace",
    ngrams=None,
    output_sequence_length=None,
    output_mode='int'
)

In [10]:
textvectorizer=TextVectorization(
    max_tokens=10000,
    output_mode='int',
    output_sequence_length=15)

# adapt() method
textvectorizer.adapt(train_sent)
# to get the unique words in vocab
wordss= textvectorizer.get_vocabulary()
len(wordss)

10000

Embedding

In [11]:
from tensorflow.keras import layers
tf.random.set_seed(42)
embeddingss= layers.Embedding(
    input_dim=10000,
    output_dim=128,
    input_length=15

)



Model 0: Naive Bayes (baseline)

Model 1: Feed-forward neural network (dense model)

Model 2: LSTM model

Model 3: GRU model

Model 4: Bidirectional-LSTM model

Model 5: 1D Convolutional Neural Network

Model 6: TensorFlow Hub Pretrained Feature Extractor

Model 7: Same as model 6 with 10% of training data

Model 0

In [12]:

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

model_0= Pipeline([
    ('tfidf',TfidfVectorizer()), #convert words to numbers using tfidf
    ("clf",MultinomialNB())

])

model_0.fit(train_sent,train_result)
baseline_score= model_0.score(validation_sent,validation_result)
baseline_pred=model_0.predict(validation_sent)

In [13]:

from sklearn.metrics import accuracy_score,precision_recall_fscore_support


def calculate_result(y_test,y_pred):
  accuracyscore=accuracy_score(y_test,y_pred)*100
  model_precision, model_recall, model_f1 ,_= precision_recall_fscore_support(y_test,y_pred,average="weighted")
  all= {
      "accuracy":accuracyscore,
      "model_precision":model_precision,
      "model_recall" : model_recall,
      "model_f1": model_f1}
  return all

In [14]:
baseline_results=calculate_result(validation_result,baseline_pred)

In [15]:
baseline_results

{'accuracy': 79.97373604727511,
 'model_precision': 0.8102693639254774,
 'model_recall': 0.7997373604727511,
 'model_f1': 0.7932459520374361}

In [16]:
import datetime
def create_tensorboard_callback(dir_name, experiment_name):

  log_dir = dir_name + "/" + experiment_name + "/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
  tensorboard_callback = tf.keras.callbacks.TensorBoard(
      log_dir=log_dir
  )
  print(f"Saving TensorBoard log files to: {log_dir}")
  return tensorboard_callback

In [17]:
SAVE_DIR="model_logs"

In [18]:
from tensorflow.keras import layers
inputs= layers.Input(shape=(1,),dtype=tf.string)
x=textvectorizer(inputs)
x=embeddingss(x)
x = layers.GlobalAveragePooling1D()(x)
output=layers.Dense(1,activation='sigmoid')(x)
model1=tf.keras.Model(inputs,output,name="model_1_dense")

In [19]:
model1.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss="binary_crossentropy",
    metrics=['accuracy'],
)

In [20]:
model1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [21]:
history_1=model1.fit(
    x=train_sent,
    y=train_result,
    epochs=5,
    validation_data=(validation_sent,validation_result),
    callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,experiment_name="simple_dense_model")])


Saving TensorBoard log files to: model_logs/simple_dense_model/20230705-105710
Epoch 1/5
191/191 [==============================] - 27s 97ms/step - loss: 0.6210 - accuracy: 0.6867 - val_loss: 0.5417 - val_accuracy: 0.7669
Epoch 2/5
191/191 [==============================] - 5s 23ms/step - loss: 0.4544 - accuracy: 0.8112 - val_loss: 0.4640 - val_accuracy: 0.8024
Epoch 3/5
191/191 [==============================] - 4s 19ms/step - loss: 0.3548 - accuracy: 0.8571 - val_loss: 0.4462 - val_accuracy: 0.8076
Epoch 4/5
191/191 [==============================] - 2s 8ms/step - loss: 0.2881 - accuracy: 0.8885 - val_loss: 0.4475 - val_accuracy: 0.8056
Epoch 5/5
191/191 [==============================] - 1s 6ms/step - loss: 0.2391 - accuracy: 0.9125 - val_loss: 0.4614 - val_accuracy: 0.7971


In [22]:
model1.evaluate(validation_sent,validation_result)

48/48 [==============================] - 0s 3ms/step - loss: 0.4614 - accuracy: 0.7971


[0.4614374041557312, 0.7971109747886658]

In [23]:
prediction_value =model1.predict(validation_sent)

48/48 [==============================] - 0s 2ms/step


In [24]:
prediction_value=tf.squeeze(tf.round(prediction_value))

In [25]:
result_model1=calculate_result(validation_result,prediction_value)

In [26]:
a=textvectorizer(["hello I'm Rushikesh parab machine learning engineer"])

In [27]:
embeddingss(a)

<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-0.03867469,  0.02980481,  0.00025772, ..., -0.00201367,
         -0.04063153, -0.00345522],
        [-0.11791833,  0.11092636, -0.13148889, ...,  0.05321972,
         -0.06867606, -0.15036285],
        [-0.00495112,  0.04552926,  0.01349617, ...,  0.0384769 ,
          0.04419824,  0.04143978],
        ...,
        [-0.04950759,  0.04593842, -0.03879189, ...,  0.03214112,
         -0.04697413,  0.00926058],
        [-0.04950759,  0.04593842, -0.03879189, ...,  0.03214112,
         -0.04697413,  0.00926058],
        [-0.04950759,  0.04593842, -0.03879189, ...,  0.03214112,
         -0.04697413,  0.00926058]]], dtype=float32)>

In [28]:
weights=embeddingss.weights[0]

In [29]:

# # Code below is adapted from: https://www.tensorflow.org/tutorials/text/word_embeddings#retrieve_the_trained_word_embeddings_and_save_them_to_disk
# import io

# # Create output writers
# out_v = io.open("embedding_vectors.tsv", "w", encoding="utf-8")
# out_m = io.open("embedding_metadata.tsv", "w", encoding="utf-8")

# # Write embedding vectors and words to file
# for num, word in enumerate(wordss):
#   if num == 0:
#      continue # skip padding token
#   vec = weights[num]
#   out_m.write(word + "\n") # write words to file
#   out_v.write("\t".join([str(x) for x in vec]) + "\n") # write corresponding word vector to file
# out_v.close()
# out_m.close()

# # Download files locally to upload to Embedding Projector
# try:
#   from google.colab import files
# except ImportError:
#   pass
# else:
#   files.download("embedding_vectors.tsv")
#   files.download("embedding_metadata.tsv")




Model2

In [30]:
input=tf.keras.Input(shape=(1,),dtype="string")
x=textvectorizer(input)
x=embeddingss(x)
print(x.shape)
x = layers.LSTM(64)(x)
print(x.shape)
outputs = layers.Dense(1, activation="sigmoid")(x)
model2 = tf.keras.Model(input, outputs, name="model_2_LSTM")

(None, 15, 128)
(None, 64)


In [31]:
model2.compile(optimizer="Adam", loss="binary_crossentropy", metrics=["accuracy"])

In [32]:
model2.summary()

Model: "model_2_LSTM"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 lstm (LSTM)                 (None, 64)                49408     
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1,329,473
Trainable params: 1,329,473
Non-trainable params: 0
____________________________________________

In [33]:
model2_history=model2.fit(train_sent,train_result,epochs=5,validation_data=(validation_sent,validation_result))

Epoch 1/5
191/191 [==============================] - 14s 58ms/step - loss: 0.2230 - accuracy: 0.9230 - val_loss: 0.5817 - val_accuracy: 0.7925
Epoch 2/5
191/191 [==============================] - 3s 16ms/step - loss: 0.1489 - accuracy: 0.9448 - val_loss: 0.5984 - val_accuracy: 0.7774
Epoch 3/5
191/191 [==============================] - 4s 22ms/step - loss: 0.1155 - accuracy: 0.9585 - val_loss: 0.7117 - val_accuracy: 0.7827
Epoch 4/5
191/191 [==============================] - 2s 9ms/step - loss: 0.0987 - accuracy: 0.9639 - val_loss: 0.7665 - val_accuracy: 0.7676
Epoch 5/5
191/191 [==============================] - 2s 8ms/step - loss: 0.0758 - accuracy: 0.9713 - val_loss: 1.1187 - val_accuracy: 0.7577


In [34]:
model_2_pred_probs = model2.predict(validation_sent)

48/48 [==============================] - 1s 3ms/step


In [35]:
model_2_pred_probs.shape

(1523, 1)

In [36]:
model_2_preds = tf.squeeze(tf.round(model_2_pred_probs))
model_2_preds[:10]


<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 0., 0., 0., 1., 0., 0., 0., 0., 1.], dtype=float32)>

In [37]:
result_model2=calculate_result(validation_result,model_2_preds)

Model3

In [38]:

inputs = layers.Input(shape=(1,), dtype="string")
x = textvectorizer(inputs)
x = embeddingss(x)
# x = layers.GRU(64, return_sequences=True) # stacking recurrent cells requires return_sequences=True
x = layers.GRU(64)(x)
# x = layers.Dense(64, activation="relu")(x) # optional dense layer after GRU cell
outputs = layers.Dense(1, activation="sigmoid")(x)
model3 = tf.keras.Model(inputs, outputs, name="model_3_GRU")

In [39]:
model3.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])


In [40]:
model3.summary()

Model: "model_3_GRU"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 gru (GRU)                   (None, 64)                37248     
                                                                 
 dense_2 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1,317,313
Trainable params: 1,317,313
Non-trainable params: 0
_____________________________________________

In [41]:
model3_history=model3.fit(train_sent,train_result,epochs=5,validation_data=(validation_sent,validation_result))


Epoch 1/5
191/191 [==============================] - 14s 54ms/step - loss: 0.1594 - accuracy: 0.9333 - val_loss: 0.8630 - val_accuracy: 0.7735
Epoch 2/5
191/191 [==============================] - 6s 29ms/step - loss: 0.0749 - accuracy: 0.9731 - val_loss: 0.7526 - val_accuracy: 0.7735
Epoch 3/5
191/191 [==============================] - 3s 17ms/step - loss: 0.0623 - accuracy: 0.9778 - val_loss: 0.8884 - val_accuracy: 0.7617
Epoch 4/5
191/191 [==============================] - 2s 10ms/step - loss: 0.0564 - accuracy: 0.9785 - val_loss: 1.0815 - val_accuracy: 0.7610
Epoch 5/5
191/191 [==============================] - 1s 8ms/step - loss: 0.0478 - accuracy: 0.9805 - val_loss: 1.1294 - val_accuracy: 0.7485


In [42]:
model_3_pred_probs = model3.predict(validation_sent)
model_3_preds = tf.squeeze(tf.round(model_3_pred_probs))

48/48 [==============================] - 0s 3ms/step


In [43]:
result_model3= calculate_result(validation_result,
                                    model_3_preds)

Model 4

In [44]:
# Build a Bidirectional RNN in TensorFlow
inputs = layers.Input(shape=(1,), dtype="string")
x = textvectorizer(inputs)
x = embeddingss(x)
# x = layers.Bidirectional(layers.LSTM(64, return_sequences=True))(x) # stacking RNN layers requires return_sequences=True
x = layers.Bidirectional(layers.LSTM(64))(x) # bidirectional goes both ways so has double the parameters of a regular LSTM layer
outputs = layers.Dense(1, activation="sigmoid")(x)
model4 = tf.keras.Model(inputs, outputs, name="model_4_Bidirectional")

In [45]:
model4.summary()

Model: "model_4_Bidirectional"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 bidirectional (Bidirectiona  (None, 128)              98816     
 l)                                                              
                                                                 
 dense_3 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1,378,945
Trainable params: 1,3

In [46]:
model4.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [47]:
mode4_history=model4.fit(train_sent,train_result,epochs=5,validation_data=(validation_sent,validation_result))

Epoch 1/5
191/191 [==============================] - 16s 62ms/step - loss: 0.0991 - accuracy: 0.9757 - val_loss: 0.9328 - val_accuracy: 0.7584
Epoch 2/5
191/191 [==============================] - 4s 18ms/step - loss: 0.0494 - accuracy: 0.9775 - val_loss: 1.0522 - val_accuracy: 0.7577
Epoch 3/5
191/191 [==============================] - 4s 21ms/step - loss: 0.0394 - accuracy: 0.9814 - val_loss: 1.3075 - val_accuracy: 0.7577
Epoch 4/5
191/191 [==============================] - 2s 13ms/step - loss: 0.0398 - accuracy: 0.9808 - val_loss: 1.4565 - val_accuracy: 0.7584
Epoch 5/5
191/191 [==============================] - 2s 9ms/step - loss: 0.0346 - accuracy: 0.9834 - val_loss: 1.4633 - val_accuracy: 0.7446


In [48]:
# Make predictions with bidirectional RNN on the validation data
model_4_pred_probs = model4.predict(validation_sent)
model_4_preds = tf.squeeze(tf.round(model_4_pred_probs))


48/48 [==============================] - 1s 3ms/step


In [49]:
result_model4 = calculate_result(validation_result,model_4_preds)
result_model4

{'accuracy': 74.45830597504924,
 'model_precision': 0.7456935548906564,
 'model_recall': 0.7445830597504924,
 'model_f1': 0.7450220852554396}

Model 5

In [50]:
input= layers.Input(shape=(1,), dtype="string")
x=textvectorizer(input)
x=embeddingss(x)
print(x)
print(x.shape)
x=layers.Conv1D(filters=32, kernel_size=5, activation="relu")(x)
print(x)
print(x.shape)

x=layers.GlobalMaxPool1D()(x)
print(x)
print(x.shape)

outputs = layers.Dense(1, activation="sigmoid")(x)
model5 = tf.keras.Model(input, outputs, name="model_5_Conv1D")


KerasTensor(type_spec=TensorSpec(shape=(None, 15, 128), dtype=tf.float32, name=None), name='embedding/embedding_lookup/Identity_1:0', description="created by layer 'embedding'")
(None, 15, 128)
KerasTensor(type_spec=TensorSpec(shape=(None, 11, 32), dtype=tf.float32, name=None), name='conv1d/Relu:0', description="created by layer 'conv1d'")
(None, 11, 32)
KerasTensor(type_spec=TensorSpec(shape=(None, 32), dtype=tf.float32, name=None), name='global_max_pooling1d/Max:0', description="created by layer 'global_max_pooling1d'")
(None, 32)


In [51]:
model5.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])


In [52]:
model5.summary()

Model: "model_5_Conv1D"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 conv1d (Conv1D)             (None, 11, 32)            20512     
                                                                 
 global_max_pooling1d (Globa  (None, 32)               0         
 lMaxPooling1D)                                                  
                                                                 
 dense_4 (Dense)             (None, 1)              

In [53]:
model_5_history = model5.fit(train_sent,train_result,epochs=5,validation_data=(validation_sent,validation_result))

Epoch 1/5
191/191 [==============================] - 17s 56ms/step - loss: 0.1200 - accuracy: 0.9677 - val_loss: 0.9277 - val_accuracy: 0.7505
Epoch 2/5
191/191 [==============================] - 4s 18ms/step - loss: 0.0658 - accuracy: 0.9736 - val_loss: 1.0817 - val_accuracy: 0.7531
Epoch 3/5
191/191 [==============================] - 3s 18ms/step - loss: 0.0520 - accuracy: 0.9777 - val_loss: 1.1784 - val_accuracy: 0.7466
Epoch 4/5
191/191 [==============================] - 2s 8ms/step - loss: 0.0468 - accuracy: 0.9811 - val_loss: 1.2819 - val_accuracy: 0.7479
Epoch 5/5
191/191 [==============================] - 1s 7ms/step - loss: 0.0448 - accuracy: 0.9823 - val_loss: 1.2921 - val_accuracy: 0.7354


In [54]:
model_5_pred_probs = model5.predict(validation_sent)
print(model_5_pred_probs.shape)
model_5_preds = tf.squeeze(tf.round(model_5_pred_probs))



48/48 [==============================] - 0s 2ms/step
(1523, 1)


In [55]:
result_model5 = calculate_result(validation_result,model_5_preds)
result_model5

{'accuracy': 73.53906762967827,
 'model_precision': 0.7377086906248691,
 'model_recall': 0.7353906762967827,
 'model_f1': 0.7361612945216657}

In [56]:
import tensorflow_hub as hub
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
embed_samples = embed([
                      "When you call the universal sentence encoder on a sentence, it turns it into numbers."])

print(embed_samples.shape)

(1, 512)


In [57]:
import tensorflow_hub as hub


sentence_encoder_layer = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4",
                                        input_shape=[],
                                        dtype=tf.string,
                                        trainable=False,
                                        name="USE")

In [58]:
model6 = tf.keras.Sequential([
    sentence_encoder_layer,
    layers.Dense(64,activation="relu"),
    layers.Dense(1, activation="sigmoid")
    ], name="model_6_USE")

model6.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])
model6.summary()

Model: "model_6_USE"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 USE (KerasLayer)            (None, 512)               256797824 
                                                                 
 dense_5 (Dense)             (None, 64)                32832     
                                                                 
 dense_6 (Dense)             (None, 1)                 65        
                                                                 
Total params: 256,830,721
Trainable params: 32,897
Non-trainable params: 256,797,824
_________________________________________________________________


In [59]:
model_6_history = model6.fit(train_sent,
                              train_result,
                              epochs=5,
                              validation_data=(validation_sent, validation_result)
                             )

Epoch 1/5
191/191 [==============================] - 6s 16ms/step - loss: 0.5108 - accuracy: 0.7852 - val_loss: 0.4312 - val_accuracy: 0.8076
Epoch 2/5
191/191 [==============================] - 3s 18ms/step - loss: 0.4204 - accuracy: 0.8146 - val_loss: 0.4116 - val_accuracy: 0.8188
Epoch 3/5
191/191 [==============================] - 4s 20ms/step - loss: 0.4044 - accuracy: 0.8213 - val_loss: 0.4127 - val_accuracy: 0.8221
Epoch 4/5
191/191 [==============================] - 3s 14ms/step - loss: 0.3935 - accuracy: 0.8294 - val_loss: 0.4119 - val_accuracy: 0.8207
Epoch 5/5
191/191 [==============================] - 3s 14ms/step - loss: 0.3871 - accuracy: 0.8287 - val_loss: 0.4163 - val_accuracy: 0.8194


In [60]:
model_6_pred_probs = model6.predict(validation_sent)
model_6_preds = tf.squeeze(tf.round(model_6_pred_probs))

48/48 [==============================] - 1s 16ms/step


In [61]:
result_model6 = calculate_result(validation_result,model_6_preds)
result_model6

{'accuracy': 81.94353250164149,
 'model_precision': 0.8194000620700118,
 'model_recall': 0.8194353250164149,
 'model_f1': 0.8194173358332375}

In [62]:
all_model_results = pd.DataFrame({"baseline": baseline_results,
                                  "simple_dense": result_model1,
                                  "lstm": result_model2,
                                  "gru": result_model3,
                                  "bidirectional": result_model4,
                                  "conv1d": result_model5,
                                  "tf_hub_sentence_encoder": result_model6
                                  })
all_model_results = all_model_results.transpose()
all_model_results

,accuracy,model_precision,model_recall,model_f1
baseline,79.973736,0.810269,0.799737,0.793246
simple_dense,79.711097,0.796375,0.797111,0.796448
lstm,75.771504,0.760930,0.757715,0.758601
gru,74.852265,0.751787,0.748523,0.749442
bidirectional,74.458306,0.745694,0.744583,0.745022
conv1d,73.539068,0.737709,0.735391,0.736161
tf_hub_sentence_encoder,81.943533,0.819400,0.819435,0.819417


0 = Not a real diaster Tweet


1 = Real diaster Tweet

In [65]:
# Turn Tweet into string
daniels_tweet = "Life like an ensemble: take the best choices from others and make your own"

In [66]:
def predict_on_sentence(model, sentence):
  pred_prob = model.predict([sentence])
  pred_label = tf.squeeze(tf.round(pred_prob)).numpy()
  print(f"Pred: {pred_label}", "(real disaster)" if pred_label > 0 else "(not real disaster)", f"Prob: {pred_prob[0][0]}")
  print(f"Text:\n{sentence}")

In [75]:
print(train_df["text"][1],train_df["target"][1])
sample1=train_df["text"][1]

Forest fire near La Ronge Sask. Canada 1


In [80]:
sample2 = "hello I'm Machine learning enginner"

In [78]:
predict_on_sentence(model6,sample1)

1/1 [==============================] - 0s 161ms/step
Pred: 1.0 (real disaster) Prob: 0.9798776507377625
Text:
Forest fire near La Ronge Sask. Canada


In [81]:
predict_on_sentence(model6,sample2)

1/1 [==============================] - 0s 40ms/step
Pred: 0.0 (not real disaster) Prob: 0.07169391959905624
Text:
hello I'm Machine learning enginner
